In [ ]:
classes = [('Airliner', 'n02690373'),
            ('Sorrel', 'n02389026'),
            ('Jack-o’-lantern', 'n03590841')
            ('Panda', 'n02510455'),
            ('Anemone fish', 'n02607072')]

print(classes)

urls_pref = 'http://image-net.org/api/text/imagenet.synset.geturls?wnid='
images_path = 'images/'

random_seed = 42

## Download required images from ImageNet

In [ ]:
import requests
import os
import json
import random
from PIL import Image

IMAGES_NEED_PER_CLASS = 70
rand = random.Random(random_seed)

for cl in classes:
    class_path = images_path + cl[0] + '/'
    if not os.path.exists(class_path):
        os.makedirs(class_path)

    urls_res = requests.get(url=urls_pref + cl[1])
    if not urls_res.ok:
        print(cl[0] + ' GET bad response: ', urls_response)
    else:
        image_urls = urls_res.text.split(sep='\r\n')
        used_urls = []
        rand.shuffle(image_urls)
        num_saved = 0
        for image_url in image_urls:
            try:
                img_res = requests.get(url=image_url, timeout=10)
            except:
                print(image_url + ' GET failed')
                continue
                
            if not img_res.ok:
                print(image_url + ' GET bad response: ', img_res)
            else:
                img_data = img_res.content
                img_path = class_path + str(num_saved) + '.jpg'
                with open(img_path, 'wb') as handler:
                    handler.write(img_data)
                    
                try:
                    original_image = Image.open(img_path)
                    num_saved += 1
                    used_urls.append(image_url)
                except:
                    print(image_url + ' corrupted, path: ' + img_path)
                    os.remove(img_path)
                    continue
                
                if num_saved == IMAGES_NEED_PER_CLASS:
                    break
            
        with open(class_path + 'used_image_urls.json', 'w', encoding='utf-8') as f:
            json.dump(used_urls, f, ensure_ascii=False, indent=4)
            
        with open(class_path + 'all_image_urls.json', 'w', encoding='utf-8') as f:
            json.dump(image_urls, f, ensure_ascii=False, indent=4)
                    
        if num_saved != IMAGES_NEED_PER_CLASS:
            print('Failed to download enough images for class ' + cl[0] + ' only ' + str(num_saved) + '/' + IMAGES_NEED_PER_CLASS)
        else:
            print(cl[0] + ' images downloading complite')
        
